# **Training the CNN**

Objectives: Preprocess the cherry leaves dataset for model training.

Inputs: Cherry leaves dataset downloaded from Kaggle.

Outputs: Preprocessed dataset ready for model training.

# Steps for training the model:

**Important Note**: After installing the required packages, you may need to restart the Jupyter Notebook kernel for the changes to take effect. This ensures that the newly installed libraries are properly loaded. You can restart the kernel by selecting "Kernel" > "Restart" in the Jupyter Notebook menu.

## 1. Import libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

## 3. Constructing the model CNN

In this step, we are extracting the cherry leaves dataset from a compressed .zip file. This action makes the dataset accessible for further processing. It involves opening the .zip file and extracting its contents into a designated directory, followed by removing the .zip file to clean up our workspace.


In [ ]:
os.chdir('/workspaces/Mildew-Detection-in-Cherry-Leaves')
print("Working directory changed to '/workspaces/Mildew-Detection-in-Cherry-Leaves'")

In [ ]:
img_width, img_height = 50, 50  # Adjusted image size
batch_size = 20  # Adjusted batch size

model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)))
model.add(MaxPooling2D(2, 2))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(2, 2))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

## 4. Compiling the model

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

## 5. Training the model

### Re-Defining Parameters

It's important to redefine the parameters if they were not defined in a previous Jupyter Notebook session. Variables defined in one Jupyter Notebook session are not automatically carried over to another session unless explicitly saved and loaded.

### Parameters

In [ ]:
train_dir = 'inputs/mildew_dataset/cherry-leaves/train'
validation_dir = 'inputs/mildew_dataset/cherry-leaves/validation'
test_dir = 'inputs/mildew_dataset/cherry-leaves/test'

### Data generators

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory( train_dir, target_size=(img_width, img_height), batch_size=batch_size, class_mode='binary')
validation_generator = validation_datagen.flow_from_directory( validation_dir, target_size=(img_width, img_height), batch_size=batch_size, class_mode='binary')
test_generator = test_datagen.flow_from_directory( test_dir, target_size=(img_width, img_height), batch_size=batch_size, class_mode='binary')

### Model training

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=15,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size)

## 6. Evaluating the model

In [ ]:
model.evaluate(test_generator)

## 7.Save the Trained Model

After successfully training our CNN, it's crucial to save the model. This enables us to use the model later for making predictions, either in a different environment or as part of an application. We will save the model in the HDF5 format, which is a versatile container format used for storing a model with its architecture, weights, training configuration, and even optimizer state.


In [ ]:
model.save('cherry_leaf_cnn_model.h5')

print("Model saved successfully.")

## 8.Cherry Leaf Condition Analysis Using CNN

In this section, we provide a straightforward interface for you to use our Convolutional Neural Network (CNN) model to analyze cherry leaves. Simply upload your images, and the model will classify them as either 'Healthy' or 'Powdery Mildew'. This tool is designed to assist in the rapid assessment of cherry leaf conditions, offering a glimpse into the power of machine learning in agricultural applications.

Please upload your cherry leaf images in JPG format, either one by one or as a ZIP file containing multiple images. The model will process each image and present you with its findings.

In [ ]:
from IPython.display import display
from PIL import Image
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import ipywidgets as widgets
from io import BytesIO
import zipfile

# Load the pre-trained CNN model
model = load_model('cherry_leaf_cnn_model.h5')

# Function to process and predict an image
def process_and_predict_image(uploaded_file):
    try:
        # Read and process the image
        img = Image.open(uploaded_file)
        img = img.resize((50, 50))
        img_array = image.img_to_array(img) / 255.0
        img_array = np.expand_dims(img_array, axis=0)

        # Predict the condition of the cherry leaf
        prediction = model.predict(img_array)
        label = 'Healthy' if prediction < 0.5 else 'Powdery Mildew'
        display(img)
        print(f'Prediction: {label}')
    except Exception as e:
        print(f'Error processing image: {e}')

# File upload widget for images and ZIP files
uploader = widgets.FileUpload(accept='image/jpeg, application/zip', multiple=True)
display(uploader)

# Button to process the uploaded images
process_button = widgets.Button(description="Analyze Images")

# Function to handle the button click event
def on_button_clicked(b):
    # Process each uploaded file
    for name, file_info in uploader.value.items():
        if name.lower().endswith('.zip'):
            # Extract and process images from ZIP file
            with zipfile.ZipFile(BytesIO(file_info['content'])) as zfile:
                for filename in zfile.namelist():
                    with zfile.open(filename) as file:
                        process_and_predict_image(file)
        else:
            # Process single images
            process_and_predict_image(BytesIO(file_info['content']))

# Register the click event handler
process_button.on_click(on_button_clicked)
display(process_button)